<a href="https://colab.research.google.com/github/petrmiculek/cnn_matchboxes/blob/master/keras_tuner_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# !pip install -U keras-tuner
# stdlib
import os
import sys

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import datetime
from itertools import product
from contextlib import redirect_stdout

import pathlib
import glob


# external libs
import tensorboard
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import Image, display
from tensorflow.python.framework.errors_impl import NotFoundError

from tensorflow.keras.callbacks import \
    TensorBoard, ReduceLROnPlateau, \
    LearningRateScheduler, ModelCheckpoint

from tensorflow.keras.layers import \
    add, Conv2D, BatchNormalization, Softmax, Input, MaxPool2D, Cropping2D, Concatenate, AvgPool2D, ZeroPadding2D
from tensorflow.keras.layers.experimental.preprocessing import \
    CenterCrop, RandomFlip, RandomRotation

from tensorboard.plugins.hparams import api as hp
import kerastuner as kt



Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.7/dist-packages (1.0.2)


In [2]:
import tensorboard

from google.colab import drive
gdrive = os.getcwd() + '/drive/MyDrive/sirky'
# drive.mount(gdrive)

Mounted at /content/drive


In [8]:
gdrive = os.getcwd() + '/drive/MyDrive/sirky'
print(os.listdir(gdrive))

['puvodni-fotky', 'chybne-klasifikovane_64_100', 'image_regions_64_050', 'full_res_heatmaps', 'tff_width128_layers18_augTrue_graph.png', 'heatmaps_tff-width128-layers18-augTrue', 'tff_w128_l1820210301113431_architecture_graph.png', 'tff_w128_l1820210301113431_layer_activations', 'residual_20210304204516_full', 'residual_20210305142236_full-bez_augmentaci-nejlepe_natrenovany', 'dilated20210306203948_full-nejvyssi-validation-accuracy', 'dilated20210308100551_full', 'residual_64x2021-03-15-01-11-41_full', '64x_050s_500bg.zip', '64x_050s_500bg.zip (Unzipped Files)']


In [9]:
target = '64x_050s_500bg.zip'
# !unzip -o /content/drive/MyDrive/sirky/64x_050s_500bg.zip

Výstupní stream byl oříznut na posledních 5000 řádků.
  inflating: 64x_050s_500bg_val/background/20201020_121235_(3098,566).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(1578,454).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(794,476).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(70,88).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(506,264).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(1940,1362).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(3080,2622).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(3794,1826).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(1968,1630).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(494,2138).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(3144,2550).jpg  
  inflating: 64x_050s_500bg_val/background/20201020_121235_(930,2570).jpg  
  inflating: 64x_050s_500bg_val/ba

## scaffolding

In [68]:
def safestr(*args):
    """Turn string into a filename
    https://stackoverflow.com/questions/7406102/create-sane-safe-filename-from-any-unsafe-string
    :return: sanitized filename-safe string
    """
    string = str(args)
    keepcharacters = (' ', '.', '_', '-')
    return "".join(c for c in string if c.isalnum() or c in keepcharacters).rstrip().replace(' ', '_')


In [86]:

def get_checkpoint_path(path=gdrive + '/model_checkpoints'):

    files = glob.glob(os.path.join(path, '*'))
    for f in files:
        if os.path.isfile(f):
            os.remove(f)
        else:
            shutil.rmtree(f)

    return os.path.join(path, 'checkpoint')


def pred_reshape(y_pred):
    return tf.reshape(y_pred, [tf.shape(y_pred)[0], tf.shape(y_pred)[3]])


class Accu(tf.metrics.SparseCategoricalAccuracy):
    def update_state(self, y_true, y_pred, sample_weight=None):
        # reshape prediction - keep only Batch and Class-probabilities dimensions
        y_pred_reshaped = pred_reshape(y_pred)

        # make prediction fail if it is undecided (all probabilities are 1/num_classes = 0.125)
        cond = tf.expand_dims(tf.math.equal(tf.math.reduce_max(y_pred_reshaped, axis=1), 0.125), axis=1)
        y_avoid_free = tf.where(cond, tf.cast(7, dtype=tf.int64), y_true)

        return super(Accu, self).update_state(y_avoid_free, y_pred_reshaped, sample_weight)


class Precision(tf.keras.metrics.Precision):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_reshaped = pred_reshape(y_pred)
        y_pred_binary = tf.where(tf.argmax(y_pred_reshaped, axis=1) > 0, 1, 0)  # lambda: 1, lambda: 0

        # y_true is evaluated as bool => ok as it is

        return super(Precision, self).update_state(y_true, y_pred_binary, sample_weight)


class Recall(tf.keras.metrics.Recall):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_reshaped = pred_reshape(y_pred)
        y_pred_binary = tf.where(tf.argmax(y_pred_reshaped, axis=1) > 0, 1, 0)  # lambda: 1, lambda: 0

        return super(Recall, self).update_state(y_true, y_pred_binary, sample_weight)



def lr_scheduler(epoch, lr, start=10, end=150, decay=-0.10):
    """Exponential learning rate decay

    https://keras.io/api/callbacks/learning_rate_scheduler/

    :param epoch: Current epoch number
    :param lr: current learning rate
    :param start: first epoch to start LR scheduling
    :param end: last epoch of LR scheduling (constant LR after)
    :param decay: Decay rate
    :return: New learning rate
    """
    if epoch < start:
        return lr
    elif epoch > end:
        return lr
    else:
        return lr * tf.math.exp(decay)


class RandomColorDistortion(tf.keras.layers.Layer):
    """Apply multiple color-related augmentations

    Adapted from:
    https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/06_preprocessing/06e_colordistortion.ipynb

    maybe low efficiency of chained operations (jpg -> float, aug, float -> jpg)
    """
    def __init__(self,
                 brightness_delta=0.2,
                 contrast_range=(0.5, 1.5),
                 hue_delta=0.2,
                 saturation_range=(0.75, 1.25),
                 **kwargs):
        super(RandomColorDistortion, self).__init__(**kwargs)
        self.brightness = brightness_delta
        self.contrast_range = contrast_range
        self.hue = hue_delta
        self.saturation_range = saturation_range


    @tf.function
    def call(self, images, training=None):
        # if training is None:
        #     training = tf.keras.learning_phase()
        if not training:
            return images

        images = tf.image.random_contrast(images, self.contrast_range[0], self.contrast_range[1])
        images = tf.image.random_brightness(images, self.brightness)
        images = tf.image.random_hue(images, self.hue)
        images = tf.image.random_saturation(images, self.saturation_range[0], self.saturation_range[1])
        images = tf.clip_by_value(images, 0, 255)
        return images

    def get_config(self, *args, **kwargs):
        # return super(RandomColorDistortion, self).get_config(*args, **kwargs)  # baseline
        return {
            'brightness_delta': self.brightness,
            'contrast_range': self.contrast_range,
            'hue_delta': self.hue,
            'saturation_range': self.saturation_range,
        }

In [87]:
def get_class_weights(class_counts_train):
    # todo equation
    class_counts_train = np.array(class_counts_train)
    num_classes = len(class_counts_train)
    class_counts_sum = np.sum(class_counts_train)
    class_weights = class_counts_sum / (num_classes * class_counts_train)
    return dict(enumerate(class_weights))

def get_dataset(data_dir):
    """
    Get dataset, class names and class weights

    Inspired by https://www.tensorflow.org/tutorials/load_data/images

    :param data_dir:
    :return:
    """
    const_seed = 1234
    autotune = tf.data.experimental.AUTOTUNE

    def get_label(file_path):
        # convert the path to a list of path components
        parts = tf.strings.split(file_path, os.path.sep)

        # The second to last is the class-directory
        # (purpose = class-number-independent encoding)

        one_hot = parts[-2] == class_names
        # Integer encode the label
        return tf.argmax(tf.cast(one_hot, dtype='uint8'))

    def process_path(file_path):
        label = get_label(file_path)

        # load raw data
        img = tf.io.read_file(file_path)

        img = tf.image.decode_jpeg(img, channels=3)
        return img, label

    def configure_for_performance(ds):
        ds = ds.cache()
        ds = ds.shuffle(buffer_size=1024, seed=const_seed)  # reshuffle_each_iteration=True
        ds = ds.batch(batch_size)
        ds = ds.prefetch(buffer_size=autotune)
        return ds

    batch_size = 128

    dataset = tf.data.Dataset.list_files(os.path.join(data_dir, '*/*.jpg'), shuffle=True)

    """ Compile a `class_names` list from the tree structure of the files """
    class_dirs, class_counts = np.unique(np.array(sorted([item.parent for item in pathlib.Path(data_dir).glob('*/*')])), return_counts=True)
    class_names = [os.path.basename(directory) for directory in class_dirs]

    """ Load images + labels, configure """
    dataset = dataset.map(process_path, num_parallel_calls=autotune)

    class_weights = get_class_weights(class_counts)  # training set only

    dataset = configure_for_performance(dataset)

    return dataset, class_names, class_weights

## model

In [88]:


def augmentation(aug=True, crop_to=64, orig_size=64):
    aug_model = tf.keras.Sequential(name='augmentation')
    aug_model.add(Input(shape=(orig_size, orig_size, 3)))

    if aug:
        aug_model.add(RandomFlip("horizontal"))
        # aug_model.add(RandomRotation(1 / 16))  # =rot22.5°
        aug_model.add(RandomColorDistortion(brightness_delta=0.3,
                                                 contrast_range=(0.25, 1.25),
                                                 hue_delta=0.1,
                                                 saturation_range=(0.75, 1.25)))

    if crop_to != 64:
        aug_model.add(CenterCrop(crop_to, crop_to))

    if not aug and crop_to == 64:
        # no other layers, model cannot be empty
        # base Layer class == identity layer
        aug_model.add(tf.keras.layers.Layer(name='identity'))

    return aug_model


In [89]:

def dilated_64x_odd(hp):
    """
    March 15

    Dilation rate growth accounts for receptive field growth caused by pooling

    :param num_classes:
    :param name_suffix:
    :return:
    """
    he_norm = tf.keras.initializers.he_normal()
    conv_args = {
        'activation': 'relu',
        'padding': 'valid',
        'kernel_initializer': he_norm
    }
    base_width = hp.Choice('base_width', values=[16])
    tail_width = hp.Choice('tail_width', values=[128, 256])
    last_width = hp.Choice('last_width', values=[64, 128, 256])
    pool_freq = hp.Choice('pool_freq', values=[1, 2, 4])
    pool_type = hp.Choice('pool_type', values=['max, avg'])

    x = Input(shape=(None, None, 3))  # None, None
    input_layer = x

    for i, width_coef in zip([1, 3, 5, 7, 9], [2, 4, 4, 8, 8]):
        w = base_width * width_coef
        x = Conv2D(w, 3, **conv_args, dilation_rate=i)(x)
        if i % pool_freq == 0:
            if pool_type == 'max':
                x = MaxPool2D((2, 2), strides=(1, 1), padding='same')(x)
            elif pool_type == 'avg':
                x = AvgPool2D((2, 2), strides=(1, 1), padding='same')(x)
            else:
                pass
        x = BatchNormalization()(x)

    x = BatchNormalization()(x)
    x = Conv2D(tail_width, 2, **conv_args, dilation_rate=11)(x)  # -> 3x3

    x = BatchNormalization()(x)
    x = Conv2D(tail_width, 3, **conv_args)(x)  # fit-once

    x = BatchNormalization()(x)
    x = Conv2D(last_width, 1, **conv_args)(x)

    x = BatchNormalization()(x)
    x = Conv2D(8, 1, kernel_initializer=he_norm, activation=None)(x)

    x = Softmax()(x)

    model = tf.keras.Model(inputs=input_layer, outputs=x,
                           name='dilated_64x_odd')
    return model, 64


In [90]:

def compile_model(model):
    """ Custom loss and metrics """
    scce_loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
    accu = Accu(name='accu')  # ~= SparseCategoricalAccuracy
    prec = Precision(name='prec')
    recall = Recall(name='recall')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=scce_loss,
        metrics=[accu,
                 prec,
                 recall,
                 ])

def get_callbacks(model, checkpoint_path=gdrive + '/checkpoint', bg_samples='_unknown'):
    """ TensorBoard loggging """
    logs_dir = os.path.join('logs', f'bg{bg_samples}', 'colab_logs')
    os.makedirs(logs_dir, exist_ok=True)
    file_writer = tf.summary.create_file_writer(logs_dir + "/metrics")
    file_writer.set_as_default()

    """ Callbacks """
    reduce_lr = ReduceLROnPlateau(monitor='accu', factor=0.5,
                                  patience=10, min_lr=5e-6)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accu', patience=15)

    # ^ No improvement on training data for 10 epochs -> Reduce LR,
    # No improvement on validation data for 15 epochs -> Halt

    tensorboard_callback = TensorBoard(logs_dir, histogram_freq=1, profile_batch='100,1100')
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        monitor='val_accu',
        mode='max',
        save_best_only=True)

    callbacks = [
        tensorboard_callback,
        early_stopping,
        reduce_lr,
        model_checkpoint_callback
    ]

    return callbacks


def build_new_model(hparams):

    base_model, training_dim = dilated_64x_odd(hparams)
    data_augmentation = augmentation(aug=True, crop_to=training_dim)
    model = tf.keras.Sequential([data_augmentation, base_model], name=base_model.name + '_full')
    compile_model(model)

    return model

## Run

In [91]:
show = False
dim = 64  # training sample dim - not really
scale = 0.5

# per image background samples

bg_samples = 500

# data_dir = f'image_regions_{dim}_{int(100 * scale):03d}_bg{bg_samples}'
data_dir = f'{64}x_{50:03d}s_{500}bg'
checkpoint_path = get_checkpoint_path()

time = safestr(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

""" Load dataset """
train_ds, class_names, class_weights = get_dataset(data_dir)
val_ds, _, _ = get_dataset(data_dir + '_val')
num_classes = len(class_names)
       

In [92]:
""" Create a model """

# model = build_new_model(None)
tuner = kt.Hyperband(
  build_new_model,
  kt.Objective("val_accu", direction="max"),
  max_epochs=50,
  hyperband_iterations=2)


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [ ]:

""" Model outputs dir """
output_location = os.path.join(gdrive, 'outputs', model.name)
if not os.path.isdir(output_location):
    os.makedirs(output_location, exist_ok=True)

# stdout_orig = sys.stdout
# out_stream = open(os.path.join(output_location, 'stdout.txt'), 'a')
# sys.stdout = DuplicateStream(sys.stdout, out_stream)

In [63]:
callbacks = get_callbacks(None, checkpoint_path, bg_samples)
print(callbacks)

[<tensorflow.python.keras.callbacks.TensorBoard object at 0x7f044d510890>, <tensorflow.python.keras.callbacks.EarlyStopping object at 0x7f044c6a3850>, <tensorflow.python.keras.callbacks.ReduceLROnPlateau object at 0x7f044d063050>, <tensorflow.python.keras.callbacks.ModelCheckpoint object at 0x7f044cff8a10>]


In [ ]:
# epochs_trained = 0
# epochs = 100

""" Train the model"""
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     validation_freq=5,
#     epochs=(epochs + epochs_trained),
#     initial_epoch=epochs_trained,
#     callbacks=callbacks,
#     class_weight=class_weights,
#     verbose=2  # one line per epoch
# )

tuner.search(train_ds,
      validation_data=val_ds,
      )

# epochs_trained += epochs

# model.load_weights(checkpoint_path)  # checkpoint

# base_model.save_weights(os.path.join(gdrive, 'models_saved', model.name))

## bs